In [34]:
# Importamos la librerías necesarias
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import db_dtypes
from transformers import pipeline
from sklearn.cluster import KMeans
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import numpy as np
import folium
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from IPython.display import display


In [2]:
# Configurar el cliente de BigQuery
client = bigquery.Client.from_service_account_json('../proyectohenry2-6383bed59f61.json')

# Cargar los datos desde BigQuery
query = """
SELECT * FROM `henry-gmaps-yelp-423106.DB_Gmaps_Yelp.Modelo`
"""
df = client.query(query).to_dataframe()

In [3]:
# Guardamos el DataFrame en formato Pickle para posteriores lecturas
df.to_pickle('../1_data/processed/Modelo.pickle')

In [4]:
df.head()

,business_id,business_name,city,state,latitude,longitude,business_stars,price,business_platform,review_user_id,...,OutdoorSeating,BusinessAcceptsCreditCards,GoodForKids,RestaurantsPriceRange2,RestaurantsTakeOut,RestaurantsReservations,HasTV,user_name,review_count,user_average_stars
0,0oSSjekU-3GR8gselReWnA,Butcher and Singer,Philadelphia,Tennessee,39.949334,-75.166176,4.5,SIN DATO,2,w6xAFxD1e15G7VMwb1mjNg,...,True,True,False,4,True,True,False,Amy,10,4.50
1,0oSSjekU-3GR8gselReWnA,Butcher and Singer,Philadelphia,Tennessee,39.949334,-75.166176,4.5,SIN DATO,2,7He2tF83YGi8cgbPXByEUg,...,True,True,False,4,True,True,False,Jennifer,257,3.89
2,0oSSjekU-3GR8gselReWnA,Butcher and Singer,Philadelphia,Tennessee,39.949334,-75.166176,4.5,SIN DATO,2,riqVnqzfWKfE37DV0LJ-gA,...,True,True,False,4,True,True,False,Quin,959,4.05
3,cecSMPG_i3sZJVDv5Mwr4g,Palm Restaurant,Tampa,Pennsylvania,27.945201,-82.527294,4.0,SIN DATO,2,xi4j3ExmHIR2RQeNggFctw,...,True,True,True,4,True,True,True,Ryan,118,4.03
4,crjRqGzxe6-MM4ylqlSfmw,Mayfield Dinner Theatre,Edmonton,Tennessee,53.554724,-113.607947,4.5,SIN DATO,2,2feiZFzICp_mLONEpzY0pA,...,False,None,False,4,False,True,True,George,364,3.91


In [6]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5596594 entries, 0 to 5596593
Data columns (total 26 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   business_id                 object        
 1   business_name               object        
 2   city                        object        
 3   state                       object        
 4   latitude                    float64       
 5   longitude                   float64       
 6   business_stars              float64       
 7   price                       object        
 8   business_platform           Int64         
 9   review_user_id              object        
 10  review_date                 datetime64[us]
 11  review_stars                Int64         
 12  review_text                 object        
 13  review_platform             Int64         
 14  category_name               object        
 15  RestaurantsDelivery         object        
 16  OutdoorSeating    

In [9]:
df.describe()

,latitude,longitude,business_stars,business_platform,review_date,review_stars,review_platform,review_count,user_average_stars
count,5.596594e+06,5.596594e+06,5.596594e+06,5596594.0,5544126,5544126.0,5544126.0,2603871.0,2.603871e+06
mean,3.533090e+01,-8.986829e+01,3.958119e+00,1.465283,2018-03-08 20:33:39.209069,3.98911,1.469663,117.292711,3.782431e+00
min,2.454690e+01,-1.242649e+02,1.000000e+00,1.0,2010-01-01 00:00:00,1.0,1.0,0.0,1.000000e+00
25%,2.996859e+01,-9.677258e+01,3.700000e+00,1.0,2017-01-15 02:35:48,3.0,1.0,8.0,3.440000e+00
50%,3.605619e+01,-8.607924e+01,4.000000e+00,1.0,2018-09-02 14:23:27,5.0,1.0,27.0,3.900000e+00
75%,3.995417e+01,-7.596512e+01,4.400000e+00,2.0,2019-11-02 00:03:09,5.0,2.0,98.0,4.290000e+00
max,5.367920e+01,-7.191950e+01,5.000000e+00,2.0,2021-12-31 23:46:27,5.0,2.0,17473.0,5.000000e+00
std,5.258615e+00,1.534625e+01,5.475653e-01,0.498793,NaN,1.297871,0.499079,327.288523,8.054106e-01


## Transformación de datos

### Columna categoría

Se realizará un filtrado para obtener solamente los registros que nos interesan para el análisis de los restaurantes y los tipos de comida

In [11]:
# Diccionario de mapeo basado en la frecuencia de tipos de comida tanto de Google como en Yelp
mapeo_categorias = {
    # Americana
    'american restaurant': 'Americana',
    'barbecue restaurant': 'Americana',
    'hamburger restaurant': 'Americana',
    'new american restaurant': 'Americana',
    'latin american restaurant': 'Americana',
    'southern restaurant (us)': 'Americana',
    'traditional american restaurant': 'Americana',
    'central american restaurant': 'Americana',
    'southern italian restaurant': 'Americana',
    'south american restaurant': 'Americana',
    'native american restaurant': 'Americana',
    'American (New)': 'Americana',
    'American (Traditional)': 'Americana',
    'Burgers': 'Americana',
    'Barbeque': 'Americana',
    'Diners': 'Americana',
    'Soul Food': 'Americana',
    'Southern': 'Americana',
    # Asiática
    'chinese restaurant': 'Asiática',
    'japanese restaurant': 'Asiática',
    'thai restaurant': 'Asiática',
    'sushi restaurant': 'Asiática',
    'asian restaurant': 'Asiática',
    'korean restaurant': 'Asiática',
    'asian fusion restaurant': 'Asiática',
    'korean barbecue restaurant': 'Asiática',
    'mongolian barbecue restaurant': 'Asiática',
    'authentic japanese restaurant': 'Asiática',
    'chinese noodle restaurant': 'Asiática',
    'southeast asian restaurant': 'Asiática',
    'pan-asian restaurant': 'Asiática',
    'delivery chinese restaurant': 'Asiática',
    'conveyor belt sushi restaurant': 'Asiática',
    'japanese curry restaurant': 'Asiática',
    'udon noodle restaurant': 'Asiática',
    'south asian restaurant': 'Asiática',
    'japanese sweets restaurant': 'Asiática',
    'japanese regional restaurant': 'Asiática',
    'kyoto style japanese restaurant': 'Asiática',
    'korean beef restaurant': 'Asiática',
    'Chinese': 'Asiática',
    'Japanese': 'Asiática',
    'Sushi Bars': 'Asiática',
    'Thai': 'Asiática',
    'Korean': 'Asiática',
    'Vietnamese': 'Asiática',
    'Asian Fusion': 'Asiática',
    # Francés
    'french restaurant': 'Francés',
    'restaurant or cafe': 'Francés',
    'modern french restaurant': 'Francés',
    'french steakhouse restaurant': 'Francés',
    'haute french restaurant': 'Francés',
    'French': 'Francés',
    'Bistros': 'Francés',
    # Hindú
    'indian restaurant': 'Hindú',
    'modern indian restaurant': 'Hindú',
    'south indian restaurant': 'Hindú',
    'north indian restaurant': 'Hindú',
    'indian muslim restaurant': 'Hindú',
    'north eastern indian restaurant': 'Hindú',
    'Indian': 'Hindú',
    'Pakistani': 'Hindú',
    # Italiana
    'pizza restaurant': 'Italiana',
    'italian restaurant': 'Italiana',
    'northern italian restaurant': 'Italiana',
    'Italian': 'Italiana',
    'Pizza': 'Italiana',
    'Pasta Shops': 'Italiana',
    # Mariscos
    'seafood restaurant': 'Mariscos',
    'fish & chips restaurant': 'Mariscos',
    'fish restaurant': 'Mariscos',
    'seafood donburi restaurant': 'Mariscos',
    'angler fish restaurant': 'Mariscos',
    'Seafood': 'Mariscos',
    'Fish & Chips': 'Mariscos',
    # Mediterránea
    'mediterranean restaurant': 'Mediterránea',
    'greek restaurant': 'Mediterránea',
    'turkish restaurant': 'Mediterránea',
    'mediterranean': 'Mediterránea',
    'Greek': 'Mediterránea',
    'Turkish': 'Mediterránea',
    'Middle Eastern': 'Mediterránea',
    'Lebanese': 'Mediterránea',
    # Mexicana
    'mexican restaurant': 'Mexicana',
    'taco restaurant': 'Mexicana',
    'burrito restaurant': 'Mexicana',
    'mexican torta restaurant': 'Mexicana',
    'Tex-Mex': 'Mexicana',
    'Tacos': 'Mexicana'
}

# Para asegurar que todos los valores están en minúsculas
mapeo_categorias = {k.lower(): v for k, v in mapeo_categorias.items()}

- Aplicamos el mapeo al dataset

In [12]:
# Asegurar que las categorías actuales están en minúsculas
df['category_name'] = df['category_name'].str.lower()

# Filtrar y renombrar las categorías basadas en el diccionario de mapeo
df['nueva_categoria'] = df['category_name'].map(mapeo_categorias)

# Eliminar filas con categorías no deseadas (que no están en el mapeo)
df = df[df['nueva_categoria'].notna()]

# Verificar el resultado
print(df[['category_name', 'nueva_categoria']].head())

             category_name nueva_categoria
0   american (traditional)       Americana
7                   french         Francés
10                 seafood        Mariscos
11          american (new)       Americana
12                 seafood        Mariscos


In [16]:
# Comprobamos la información del nuevo dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1330789 entries, 0 to 5596585
Data columns (total 27 columns):
 #   Column                      Non-Null Count    Dtype         
---  ------                      --------------    -----         
 0   business_id                 1330789 non-null  object        
 1   business_name               1330789 non-null  object        
 2   city                        1330789 non-null  object        
 3   state                       1330789 non-null  object        
 4   latitude                    1330789 non-null  float64       
 5   longitude                   1330789 non-null  float64       
 6   business_stars              1330789 non-null  float64       
 7   price                       1330789 non-null  object        
 8   business_platform           1330789 non-null  Int64         
 9   review_user_id              1323869 non-null  object        
 10  review_date                 1323869 non-null  datetime64[us]
 11  review_stars                1

- Efectivamente el filtrado por categorías nos permite optimizar el número de registros a utilizar para el sistema de recomendación al pasar de tener 5.596.594 y ahora tenemos 1.330.789 registros.

In [15]:
df['nueva_categoria'].unique().tolist()

['Americana',
 'Francés',
 'Mariscos',
 'Italiana',
 'Asiática',
 'Hindú',
 'Mexicana',
 'Mediterránea']

In [17]:
# Eliminar la columna original 'category_name'
df.drop(columns=['category_name'], inplace=True)

# Renombrar la columna 'nueva_categoria' a 'category_name'
df.rename(columns={'nueva_categoria': 'category_name'}, inplace=True)

# Verificar el resultado
print(df[['category_name']].head())

   category_name
0      Americana
7        Francés
10      Mariscos
11     Americana
12      Mariscos


- Se eliminan los atributos que no contribuyen al sistema de recomendación o que provocarían sesgo porque no están presentes en todos los registros como es el caso de los atributos binarios que aparecen en YELP pero que no aparecen en Google.

In [19]:
# Se eliminan atributos que no contribuyen a la evaluación del sistema de recomendación.
df.drop(columns=['business_name', 'price', 'review_user_id', 'review_date', 'user_name'], inplace=True)

# Se eliminarán también los campos binarios relacionados con algunos servicios adicionales que prestan los restaurantes porque están presentes solamente 
# en la mitad de los registros, es decir solamente en los registros relacionados con YELP.
df.drop(columns=['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'GoodForKids', 'RestaurantsPriceRange2', 'RestaurantsTakeOut',
                 'RestaurantsReservations', 'HasTV'], inplace=True)

# Verificamos la información del dataset resultante
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1330789 entries, 0 to 5596585
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   business_id         1330789 non-null  object 
 1   city                1330789 non-null  object 
 2   state               1330789 non-null  object 
 3   latitude            1330789 non-null  float64
 4   longitude           1330789 non-null  float64
 5   business_stars      1330789 non-null  float64
 6   business_platform   1330789 non-null  Int64  
 7   review_stars        1323869 non-null  Int64  
 8   review_text         1090521 non-null  object 
 9   review_platform     1323869 non-null  Int64  
 10  review_count        747380 non-null   Int64  
 11  user_average_stars  747380 non-null   float64
 12  category_name       1330789 non-null  object 
dtypes: Int64(4), float64(4), object(5)
memory usage: 147.2+ MB


In [22]:
# Eliminaremos también la columna review_count y la columna user_average_stars porque son estadísticas asociadas al usuario y no al negocio,
# y no serán tomadas en cuenta en el modelo.
df.drop(columns=['review_count', 'user_average_stars'], inplace=True)

### Análisis de Sentimiento

In [37]:
# Descargar los recursos necesarios de NLTK
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/diegovelez/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [38]:
# Inicializar el analizador de sentimientos de VADER
sia = SentimentIntensityAnalyzer()

# Definir una función para obtener el sentimiento de una reseña con VADER y mapearlo a una escala de 1 a 5
def obtener_sentimiento_vader(texto):
    if texto is None or texto.strip() == "":
        return 3  # Consideramos neutral si el texto está vacío o es None
    sentimiento = sia.polarity_scores(texto)
    if sentimiento['compound'] >= 0.5:
        return 5
    elif sentimiento['compound'] >= 0.1:
        return 4
    elif sentimiento['compound'] > -0.1:
        return 3
    elif sentimiento['compound'] > -0.5:
        return 2
    else:
        return 1

# Aplicar la función a las reseñas
df['sentimiento'] = df['review_text'].apply(obtener_sentimiento_vader)

# Verificar el resultado
print(df[['review_text', 'sentimiento']].head())

                                          review_text  sentimiento
0   Food was perfectly cooked. Filet mignon medium...            5
7   I'm not as impressed as most reviewers. 3 cour...            5
10  I'm not ordinarily a steakhouse fan, but Lucky...            5
11  I was invited to a company dinner here and was...            5
12  We went for my Birthday lobster & it was great...            5


In [39]:
# Eliminamos la columna review_text
df.drop(columns=['review_text'], inplace=True)

- Comprobamos los valores nulos del dataset resultante para encargarnos de que el modelo no tenga estos valores nulos

In [40]:
df.isnull().sum()

business_id             0
city                    0
state                   0
latitude                0
longitude               0
business_stars          0
business_platform       0
review_stars         6920
review_platform      6920
category_name           0
sentimiento             0
dtype: int64

In [41]:
# Manejar valores nulos
# Llenar valores nulos en 'review_stars' y 'review_platform' con la mediana
df['review_stars'].fillna(df['review_stars'].median(), inplace=True)
df['review_platform'].fillna(df['review_platform'].median(), inplace=True)

In [42]:
# Comprobamos los valores nulos nuevamente
df.isnull().sum()

business_id          0
city                 0
state                0
latitude             0
longitude            0
business_stars       0
business_platform    0
review_stars         0
review_platform      0
category_name        0
sentimiento          0
dtype: int64

In [43]:
# Guardamos el DataFrame en formato Pickle para posteriores lecturas
df.to_pickle('../1_data/processed/Dataset_filtrado.pickle')

### Checkpoint
- A partir de este momento cargaremos el dataset a partir del archivo pickle para optimizar los tiempos de procesamiento del dataset

In [62]:
# Cargamos el DataFrame desde el archivo pickle
df = pd.read_pickle('../1_data/processed/Dataset_filtrado.pickle')

- Se corregirá la columna 'state' para que corresponda al Estado dependiendo de la latitud y longitud

In [64]:
# Función para determinar el estado basado en latitud y longitud
def determinar_estado(lat, lon):
    if 32.5 <= lat <= 42 and -124 <= lon <= -114:
        return 'California'
    elif 24.5 <= lat <= 31 and -87.5 <= lon <= -80:
        return 'Florida'
    elif 39 <= lat <= 42 and -80.5 <= lon <= -74.5:
        return 'Pennsylvania'
    elif 35 <= lat <= 36.7 and -90 <= lon <= -81.5:
        return 'Tennessee'
    elif 25.8 <= lat <= 36.5 and -106.5 <= lon <= -93.5:
        return 'Texas'
    elif 40.5 <= lat <= 45 and -79.75 <= lon <= -71.75:
        return 'New York'
    else:
        return 'Unknown'


# Eliminar la columna 'city'
df.drop(columns=['city'], inplace=True)

# Asignar el estado correcto basado en latitud y longitud
df['state'] = df.apply(lambda row: determinar_estado(row['latitude'], row['longitude']), axis=1)

# Filtrar el dataset para mantener solo los registros con estados conocidos
df = df[df['state'] != 'Unknown']

# Mostrar el DataFrame actualizado y un resumen de los estados
print(df[['latitude', 'longitude', 'state']].head())
print(df['state'].value_counts())

     latitude   longitude         state
0   39.949334  -75.166176  Pennsylvania
10  34.421239 -119.641027    California
12  27.945201  -82.527294       Florida
16  27.925290  -82.515989       Florida
19  34.421239 -119.641027    California
state
Pennsylvania    268631
Florida         256560
California      177814
Texas           137642
New York        109761
Tennessee       105105
Name: count, dtype: int64


## Implementación del modelo piloto

In [65]:
# Filtrar los datos por estado y tipo de comida
# En el deploy cambiará por el Estado y la selección de tipo de comida del usuario
estado_seleccionado = 'California'
tipo_comida_seleccionado = 'Italiana'

df_filtrado = df[(df['state'] == estado_seleccionado) & (df['category_name'] == tipo_comida_seleccionado)]

In [66]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21055 entries, 851 to 5596236
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   business_id        21055 non-null  object 
 1   state              21055 non-null  object 
 2   latitude           21055 non-null  float64
 3   longitude          21055 non-null  float64
 4   business_stars     21055 non-null  float64
 5   business_platform  21055 non-null  Int64  
 6   review_stars       21055 non-null  Int64  
 7   review_platform    21055 non-null  Int64  
 8   category_name      21055 non-null  object 
 9   sentimiento        21055 non-null  int64  
dtypes: Int64(3), float64(3), int64(1), object(3)
memory usage: 1.8+ MB


### Ingeniería de atributos

In [67]:
# Calcular densidad de restaurantes
def calcular_densidad(latitudes, longitudes, radio=1.0):
    coords = np.array(list(zip(latitudes, longitudes)))
    nbrs = NearestNeighbors(radius=radio, metric='haversine').fit(np.radians(coords))
    densidad = nbrs.radius_neighbors_graph(np.radians(coords)).sum(axis=1)
    return densidad

df_filtrado['densidad_restaurantes'] = calcular_densidad(df_filtrado['latitude'], df_filtrado['longitude'])

# Calcular distancia a restaurantes populares
restaurantes_populares = df[df['business_stars'] >= 4][['latitude', 'longitude']]
coords_populares = np.array(list(zip(restaurantes_populares['latitude'], restaurantes_populares['longitude'])))
nbrs_populares = NearestNeighbors(n_neighbors=1, metric='haversine').fit(np.radians(coords_populares))
distancias, _ = nbrs_populares.kneighbors(np.radians(df_filtrado[['latitude', 'longitude']]))
df_filtrado['distancia_restaurantes_populares'] = distancias * 6371  # Convertir a kilómetros

/var/folders/j1/pdsb5w0x52sgplylm6f7fmgm0000gn/T/ipykernel_2200/4260237364.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['densidad_restaurantes'] = calcular_densidad(df_filtrado['latitude'], df_filtrado['longitude'])
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/var/folders/j1/pdsb5w0x52sgplylm6f7fmgm0000gn/T/ipykernel_2200/4260237364.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [68]:
df_filtrado.head()

,business_id,state,latitude,longitude,business_stars,business_platform,review_stars,review_platform,category_name,sentimiento,densidad_restaurantes,distancia_restaurantes_populares
851,0x808e30f8a00bc9bb:0x2259e3faa7b24b9e,California,37.821445,-121.999198,4.3,1,5,1,Italiana,5,21055.0,0.000000
3469,0x80dd304bb43ca2f7:0x9e38f711f2e4a30e,California,33.761795,-118.138818,4.4,1,5,1,Italiana,5,21055.0,0.000000
3556,0x80843e07e92c87b3:0x1cbd1bb3eebae67,California,38.549591,-122.806446,3.6,1,4,1,Italiana,5,21055.0,0.494542
3696,0x809b21fe8a3bb329:0x4479b7023fb9b6f7,California,38.792456,-121.290933,4.2,1,5,1,Italiana,5,21055.0,0.000000
3733,0x80c331c20cbee875:0xa4e1fdebd78f4866,California,34.094285,-117.721245,4.0,1,1,1,Italiana,1,21055.0,0.000000


### Escalado de atributos necesarios para el modelo

In [70]:
# Seleccionar las columnas necesarias, excluyendo latitude y longitude de la normalización
columnas_para_escalar = [
    'business_stars', 'business_platform', 'review_stars', 'review_platform', 
    'sentimiento', 'densidad_restaurantes', 'distancia_restaurantes_populares'
]

# Escalado de características numéricas (sin incluir latitude y longitude)
scaler = StandardScaler()
df_filtrado[columnas_para_escalar] = scaler.fit_transform(df_filtrado[columnas_para_escalar])

/var/folders/j1/pdsb5w0x52sgplylm6f7fmgm0000gn/T/ipykernel_2200/11948661.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado[columnas_para_escalar] = scaler.fit_transform(df_filtrado[columnas_para_escalar])


### Implementación del modelo

In [71]:
# Aplicar KMeans clustering para encontrar clusters de ubicaciones positivas, sin escalar latitud y longitud
locations = df_filtrado[['latitude', 'longitude']]
otras_caracteristicas = df_filtrado[columnas_para_escalar]

# Concatenar las características sin escalar latitud y longitud
features = pd.concat([locations.reset_index(drop=True), otras_caracteristicas.reset_index(drop=True)], axis=1)

# Parámetro para personalizar el número de centroides
num_centroides = 5  # Este valor se puede cambiar según las necesidades

kmeans = KMeans(n_clusters=num_centroides, random_state=42)
kmeans.fit(features)

# Obtener los centroides de los clusters como las ubicaciones recomendadas
centroides = kmeans.cluster_centers_

/opt/homebrew/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [72]:
# Diccionario para centrar el mapa en los estados seleccionados
centro_estados = {
    'California': [36.7783, -119.4179],
    'Florida': [27.9944024, -81.7602544],
    'Pennsylvania': [41.2033, -77.1945],
    'Tennessee': [35.5175, -86.5804],
    'Texas': [31.9686, -99.9018],
    'New York': [40.7128, -74.0060]
}

In [75]:
# Crear un mapa centrado en el estado seleccionado
centro_estado = centro_estados[estado_seleccionado]
mapa = folium.Map(location=centro_estado, zoom_start=7)

# Agregar solo los centroides (ubicaciones recomendadas) al mapa con latitud y longitud en el popup
for centroide in centroides:
    folium.Marker(
        location=[centroide[0], centroide[1]],
        popup=f'Ubicación Recomendada<br>Latitud: {centroide[0]}<br>Longitud: {centroide[1]}',
        icon=folium.Icon(color='red', icon='star')
    ).add_to(mapa)

# Mostrar el mapa en el notebook
display(mapa)

In [76]:
# Guardar el mapa en un archivo HTML
mapa.save('../8_results/' + 'mapa_recomendaciones.html')